In [1]:
import gdown
import numpy as np
import matplotlib.pyplot as plt
import os
import pyvista as pv
from collections import Counter
import natsort
pv.set_jupyter_backend('client')
pv.global_theme.trame.server_proxy_enabled = True
# pv.global_theme.trame.jupyter_extension_enabled=True


/u/qibang/.local/lib/python3.10/site-packages/pyvista/plotting/themes.py:1528: UserWarning: Enabling server_proxy will disable jupyter_extension
  warnings.warn("Enabling server_proxy will disable jupyter_extension")


In [2]:

num_cell_nodes = {
    pv.CellType.EMPTY_CELL: 0,
    pv.CellType.VERTEX: 1,
    pv.CellType.LINE: 2,
    pv.CellType.TRIANGLE: 3,
    pv.CellType.PIXEL: 4,
    pv.CellType.QUAD: 4,
    pv.CellType.TETRA: 4,
    pv.CellType.VOXEL: 8,
    pv.CellType.HEXAHEDRON: 8,
    pv.CellType.WEDGE: 6,
    pv.CellType.PYRAMID: 5,
    pv.CellType.PENTAGONAL_PRISM: 10,
    pv.CellType.HEXAGONAL_PRISM: 12,

    # Quadratic elements
    pv.CellType.QUADRATIC_EDGE: 3,
    pv.CellType.QUADRATIC_TRIANGLE: 6,
    pv.CellType.QUADRATIC_QUAD: 8,
    pv.CellType.QUADRATIC_TETRA: 10,
    pv.CellType.QUADRATIC_HEXAHEDRON: 20,
    pv.CellType.QUADRATIC_WEDGE: 15,
    pv.CellType.QUADRATIC_PYRAMID: 13,
    pv.CellType.BIQUADRATIC_QUAD: 9,
    pv.CellType.TRIQUADRATIC_HEXAHEDRON: 27,
    pv.CellType.TRIQUADRATIC_PYRAMID: 19,
    pv.CellType.QUADRATIC_LINEAR_QUAD: 6,
    pv.CellType.QUADRATIC_LINEAR_WEDGE: 12,
    pv.CellType.BIQUADRATIC_QUADRATIC_WEDGE: 18,
    pv.CellType.BIQUADRATIC_QUADRATIC_HEXAHEDRON: 24,
    pv.CellType.BIQUADRATIC_TRIANGLE: 7,
    pv.CellType.CUBIC_LINE: 4,
}


def download_data(data_path):
    # download the data from:
    # https://drive.google.com/drive/folders/1PMuOGUO2A9rzAP3YM8ra3v8ybE8X9foh
    file_names = ["car-cfd.tgz"]
    # Extract from the shared URL
    file_id = ["1SWU-b4GrfFkWUrvxgEVe4XuU9iVvhrSg"]
    for i in range(len(file_names)):
        if os.path.exists(data_path + file_names[i]):
            print(f"{file_names[i]} already exists.")
        else:
            print(f"Downloading {file_names[i]}...")
            url = f"https://drive.google.com/uc?id={file_id[i]}"
            gdown.download(url, data_path,)

    current_path = os.getcwd()
    os.chdir(data_path)
    if not os.path.exists("./training_data"):
        os.system("tar -xvzf car-cfd.tgz")
    os.chdir(current_path)


data_path = "/work/nvme/bbka/qibang/repository_WNbbka/GINTO_data/shapeNet-car/"
download_data(data_path)


car-cfd.tgz already exists.


In [3]:
params_paths = [f.path for f in os.scandir(
    data_path+"/training_data") if f.is_dir()]
params_paths = natsort.natsorted(params_paths)
samples_path = [f.path for params_p in params_paths for f in os.scandir(
    params_p) if f.is_dir()]
samples_path = natsort.natsorted(samples_path)


In [11]:
id = 4

data_base = samples_path[id]

press = np.load(data_base+"/press.npy")
mesh = pv.read(data_base + "/quadpress_smpl.vtk")
# Get the points, cells, and field data from the mesh
mesh.plot()

Widget(value='<iframe src="/proxy/38957/index.html?ui=P_0x7f0d14b4f220_6&reconnect=auto" class="pyvista" style…

In [28]:
points = mesh.points
cells = mesh.cells
field_data = mesh.point_data
cell_types = mesh.celltypes
unique_elements = np.unique(cells)
print("number of points", len(points))
print("number of cells", len(cells))
print("press shape", press.shape)

number of points 3682
number of cells 17920
press shape (3682,)


In [ ]:
def check_boundary_points(data_base):
    mesh = pv.read(data_base + "/quadpress_smpl.vtk")
    cells = mesh.cells
    unique_elements = np.unique(cells)
    return unique_elements

boundary_idxs=[check_boundary_points(samples_path[i]) for i in range(len(samples_path))]
for i in range(len(boundary_idxs)-1):
    if any(boundary_idxs[i] != boundary_idxs[i+1]):
        print("boundary points are different")
        break


In [27]:
np.savetxt(data_path+"car-pressure-data/boundary_idxs.txt", boundary_idxs[0], fmt='%d')

In [ ]:
# Find the boundary points based on the cells
num_nodes = np.array([num_cell_nodes[cell_type]
                     for cell_type in cell_types])

connectivity = []
index = 0
for n in num_nodes:
    connectivity.append(cells[index: index + n])
    index += n
unique_elements = np.unique(cells)
boundary_points = points[unique_elements]
print("number of bc point", len(boundary_points))
boundary_mesh = pv.PolyData(boundary_points)
boundary_mesh.plot()

# %
# # %%

